# dataset preprocess



In [3]:
import pandas as pd
import re

import os
from together import Together

from openai import OpenAI
import random
from collections import Counter
import time

In [2]:
!ls ./dataset

balanced_benchmark_0828_5039.csv


In [ ]:
emoji_dataset = pd.read_csv('./dataset/balanced_benchmark_0828_5039.csv')


In [ ]:
emoji_dataset.info()
emoji_dataset.index

In [ ]:
len(emoji_dataset)

In [ ]:

# Remove emoji from video_description column
emoji_dataset['sentence'] = emoji_dataset['emoji_sentence'].apply(remove_emojis)

## remove emojis to get 'sentence' column

In [ ]:
def remove_emojis(text):
    # Define a regular expression pattern for emojis and emoticons
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
                           u"\U0001F004"             # Mahjong Tile Red Dragon
                           u"\U0001F0CF"             # Playing Cards Joker
                           u"\U0001F170-\U0001F251"  # Enclosed Ideographic Supplement
                           u"\U0001F300-\U0001F320"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F321-\U0001F32C"  # Enclosed CJK Letters and Months
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002500-\U00002BEF"  # chinese char
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u200d"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\u3030"
                           u"\ufe0f"
                           u"]+", flags=re.UNICODE)
    # Remove emojis and emoticons from the text
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text


In [ ]:

# Remove emoji from video_description column
emoji_dataset['sentence'] = emoji_dataset['emoji_sentence'].apply(remove_emojis)

## extract emojis to get real_emojis column

In [ ]:

def extract_emojis(text):
    if text is None or (isinstance(text, float) and math.isnan(text)):
        return [] 
    emoji_pattern = re.compile(
        "["                              
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F700-\U0001F77F"  # Alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
        u"\U0001F004"             # Mahjong Tile Red Dragon
        u"\U0001F0CF"             # Playing Cards Joker
        u"\U0001F1E6-\U0001F1FF"  # Flags
        u"\U000025AA-\U000025AB"  # Small Squares
        u"\U000025FE-\U000025FF"  # Black Medium Squares
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U00002B50"             # Star
        u"\U0001F1E6-\U0001F1FF"  # Regional indicator symbols
        u"\u2640-\u2642"          # Gender signs
        u"\u2600-\u26FF"          # Miscellaneous Symbols
        u"\u2700-\u27BF"          # Dingbats
        "]+", flags=re.UNICODE)

    return emoji_pattern.findall(text)


In [ ]:
emojis_list=[]
for i in range(dataset_len):
    text=emoji_dataset.at[i,'emoji_sentence']
    list_emojis = extract_emojis(text)
    # print(i, list_emojis)
    str_emojis=''.join(list_emojis)
    print(i,str_emojis, len(str_emojis))
    emojis_list.append(str_emojis)

    emoji_dataset.at[i, 'real_emojis']=str_emojis
    



In [ ]:
emoji_dataset.info()

In [ ]:
dataset_len=len(emoji_dataset)
emoji_dataset.to_csv(f'./dataset/balanced_benchmark_{dataset_len}.csv', index=False)


## use judge model to get labels

the judge model is gpt-4o-mini



In [1]:
!ls ./dataset

balanced_benchmark_0828_5039.csv


In [4]:
dataset_path = './dataset/balanced_benchmark_0828_5039.csv'

emoji_dataset = pd.read_csv(dataset_path)

In [5]:
dataset_len=len(emoji_dataset)
print(dataset_len)

5039


In [6]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0.1      5039 non-null   int64 
 1   Unnamed: 0        5039 non-null   int64 
 2   index             5039 non-null   int64 
 3   user_id           5039 non-null   object
 4   emoji_sentence    5039 non-null   object
 5   char_count        5039 non-null   int64 
 6   cluster           5039 non-null   int64 
 7   sentence          5039 non-null   object
 8   gender_labels     5039 non-null   object
 9   sentiment_labels  5039 non-null   object
 10  emotion_labels    5039 non-null   object
 11  age_labels        5039 non-null   object
 12  stance_labels     5039 non-null   object
 13  is_duplicate      5039 non-null   bool  
dtypes: bool(1), int64(5), object(8)
memory usage: 516.8+ KB


In [ ]:
judge_model_name='gpt-4o-mini'
dataset_len=len(emoji_dataset)


In [ ]:
print(judge_model_name)
print(dataset_len)

In [ ]:


api_key='' # input your openai api key here

client = OpenAI(api_key=api_key)


In [ ]:
prompts = {
    "sentiment": "I will provide you a tweet. Please classify the sentiment of the tweet as positive, negative, or neutral. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "emotion": "I will provide you a tweet. Please classify the emotion of the tweet as sadness, happiness, fear, anger, surprise, or disgust. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "age": "I will provide you a tweet. Please classify the likely age group of the person who wrote the tweet as 'child', 'teen', 'adult', or 'senior'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "gender": "I will provide you a tweet. Please classify the likely gender of the person who wrote the tweet as 'male' or 'female'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "stance": "I will provide you a tweet. Please classify the stance of the tweet as 'favor', 'None', or 'against'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n"
}

def judge_classifier(judge_model_name, task, text):
    prompt = prompts[task].format(text=text)
    
    completion = client.chat.completions.create(
        model=judge_model_name,
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
    )
    prediction = completion.choices[0].message.content.strip().lower()
    return prediction

In [ ]:
print(dataset_len)

task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
start_time = time.time()  # start time

for task in task_list: 
    column_name = f'{task}_labels'
    print(f'start the {task} labels classification, the column name is {column_name}:')
    
    # Define the folder path
    folder_path = f"./dataset_with_labels/{judge_model_name}/{task}"
    os.makedirs(folder_path, exist_ok=True)

    logout_len = 200
    left_bound = 0
    
    # if task =='emotion': left_bound=2000 # !!!
    # else: left_bound=0
    
    for i in range(left_bound, dataset_len):
        # if pd.isna(emoji_dataset.at[i, column_name]):
        predicted_sentence = emoji_dataset['emoji_sentence'][i]
        temp_labels = judge_classifier(judge_model_name, task, predicted_sentence)
        print(i, predicted_sentence, '\n', temp_labels)
        
        emoji_dataset.at[i, column_name] = temp_labels
        
        if (i+1) % logout_len == 0: time.sleep(5)
            
        if (i + 1) % logout_len == 0: 
            end_time = time.time()  # end time
            elapsed_time = end_time - start_time 
            print(f"the run time of {logout_len} iterations: {elapsed_time:.4f} seconds")
                        
            output_labels_path = f"./dataset_with_labels/{judge_model_name}/{task}/dataset_labels_{judge_model_name}_{task}_{i+1}.csv"

            emoji_dataset.to_csv(output_labels_path, index=False)
            print(f"saved successfully: {output_labels_path}")

            start_time = time.time()
    output_labels_path = f"./dataset_with_labels/{judge_model_name}/{task}/dataset_labels_{judge_model_name}_{task}_{i+1}.csv"

    emoji_dataset.to_csv(output_labels_path, index=False)
    print(f"saved successfully: {output_labels_path}")
    